In [ ]:
!pip install --upgrade pip
#!pip uninstall -y nncase
!pip install nncase==1.9.0.20230322 --timeout=1000
#from versions: 1.0.0.20211029, 1.1.0.20211203, 1.3.0.20220127, 1.4.0.20220303, 1.5.0.20220331, 
# 1.6.0.20220505, 1.7.0.20220530, 1.7.1.20220701, 1.8.0.20220929, 1.9.0.20230322, 2.0.0.20230602, 2.1.0.20230703)

In [ ]:
import nncase
from nncase_base_func import *
# from parse_model import *


def compile_kmodel(model_path, dump_path, calib_data):
    """
    Set compile options and ptq options.
    Compile kmodel.
    Dump the compile-time result to 'compile_options.dump_dir'
    """
    print("----------model simplify----------")
    model_file = model_simplify(model_path)

    print("---------- set  options ----------")
    # import_options
    import_options = nncase.ImportOptions()
    
    # compile_options
    compile_options = nncase.CompileOptions()
    compile_options.target = "k210" # "cpu" "k510"
    compile_options.dump_ir = True  # if False, will not dump the compile-time result.
    compile_options.dump_asm = True
    compile_options.dump_dir = dump_path

    # preprocess args
    compile_options.preprocess = True
    if compile_options.preprocess:
        compile_options.input_type = "uint8" # "uint8"
        compile_options.swapRB = False
        compile_options.input_shape = [1,224,320,3]
        compile_options.input_range = [0,1]
        compile_options.mean = [0,0,0]
        compile_options.std = [1,1,1]
        compile_options.input_layout = "NHWC" # "NHWC"
        compile_options.output_layout = "NHWC" # "NHWC"
        compile_options.letterbox_value = 0
    
    # quant args
    compile_options.quant_type = "uint8" 
    compile_options.w_quant_type = "uint8"
    compile_options.use_mse_quant_w = True
    compile_options.split_w_to_act = False

    # quant options
    ptq_options = nncase.PTQTensorOptions()
    ptq_options.calibrate_method = "no_clip" # "kld_m2" "l2" "cdf"
    ptq_options.samples_count = len(calib_data[0])
    ptq_options.set_tensor_data(np.array(calib_data).tobytes())

    
    # set options
    compiler = nncase.Compiler(compile_options)
    compiler.use_ptq(ptq_options)
    
    print("----------   compile    ----------")
    # import
    model_content = read_model_file(model_file)
    if model_path.split(".")[-1] == "onnx":
        compiler.import_onnx(model_content, import_options)
    elif model_path.split(".")[-1] == "tflite":
        compiler.import_tflite(model_content, import_options)

    # compile
    compiler.compile()
    kmodel = compiler.gencode_tobytes()
    
    kmodel_path = os.path.join(dump_path, "test.kmodel")
    with open(kmodel_path, 'wb') as f:
        f.write(kmodel)
    print("----------  compile end ----------")
    return kmodel_path


In [ ]:
# compile kmodel single input
model_path = "./model_f32.tflite"
dump_path = "./tmp"

# If model has multi inputs, calib_data format is "[[x1, x2,...], [y1, y2,...], ...]"
# e.g. Model has three inputs (x, y, z), the calib_data is '[[x1, x2, x3],[y1, y2, y3],[z1, z2, z3]]'

calib_data = [[np.random.rand(1,224,320,3).astype(np.float32), np.random.rand(1,224,320,3).astype(np.float32)]]
kmodel_path = compile_kmodel(model_path, dump_path, calib_data)


In [ ]:
# run kmodel(simulate)
kmodel_path = "./tmp/test.kmodel"
input_data = [np.random.rand(1,224,320,3).astype(np.float32)]

result = run_kmodel(kmodel_path, input_data)
for idx, i in enumerate(result):
    print(i.shape)
    i.tofile(os.path.join(dump_path, "nncase_result_{}.bin".format(idx)))


In [ ]:
# compile kmodel multiple inputs
model_path = "./decoder_100.onnx"
dump_path = "./tmp_dec"

# If model has multiple inputs, calib_data format is "[[x1, x2,...], [y1, y2,...], ...]"
# e.g. Model has three inputs (x, y, z), the calib_data is '[[x1, x2, x3],[y1, y2, y3],[z1, z2, z3]]'

calib_data = [[np.random.randint(1, 5, size=[3, 100], dtype='int64'), np.random.randint(1, 5, size=[3, 100], dtype='int64')],
              [np.random.rand(100, 3, 192).astype(np.float32), np.random.rand(100, 3, 192).astype(np.float32)],
              [np.random.rand(3, 100).astype(np.float32) > 0.5, np.random.rand(3, 100).astype(np.float32) > 0.5], ]  # bool

kmodel_path = compile_kmodel(model_path, dump_path, calib_data)


In [ ]:
# run kmodel(simulate)
import os

kmodel_path = "./tmp_dec/test.kmodel"
input_data = [np.random.randint(1, 5, size=[3, 100], dtype='int64'),
              np.random.rand(100, 3, 192).astype(np.float32),
              np.random.rand(3, 100).astype(np.float32) > 0.5, ]

result = run_kmodel(kmodel_path, input_data)

for idx, i in enumerate(result):
    print(i.shape)
    i.tofile(os.path.join(dump_path, "nncase_result_{}.bin".format(idx)))
